In [1]:
import requests, graphviz, textwrap
from lxml import html

In [2]:
#response = requests.get("http://www.clickhole.com/clickventure/would-you-survive-bear-attack-1230")
#response = requests.get("http://www.clickhole.com/clickventure/can-you-survive-seeing-grease-broadway-4331#2,")
response = requests.get("http://www.clickhole.com/clickventure/mysterious-shadows-skullshadow-island-1880#2,")

In [3]:
#nodelayout = ""

overall_style = {
        #'label': 'A Fancy Graph',
        'fontsize': '16',
        'fontname' : 'Century Gothic Bold',
        'fontcolor': '#3a3a3a',
        'bgcolor': '#ffffff',
        #'rankdir': 'BT',
}
normal_node_style = {
        'fontname': 'Century Gothic Bold',
        'shape': 'rectangle',
        'fontcolor': '#3a3a3a',
        'color': '#3a3a3a',
        'style': 'filled',
        'fillcolor': '#efefef',
}
finish_node_style = {
        'fontname': 'Century Gothic Bold',
        'shape': 'octagon',
        'fontcolor': '#efefef',
        'color': '#983a12',
        'style': 'filled',
        'fillcolor': '#fe621e',
        'color' : '#888888'
}
start_node_style = {
        'fontname': 'Century Gothic Bold',
        'shape': 'rectangle',
        'fontcolor': '#3a3a3a',
        'color': '#983a12',
        'style': 'filled',
        'fillcolor': '#fe621e',
}
edge_style = {
        'style': 'dashed',
        'color': '#3a3a3a',
        #'arrowhead': 'open',
        #'fontname': 'Courier',
        'fontsize': '12',
        'fontcolor': '#3a3a3a',
}

In [4]:
tree = html.fromstring(response.content)

In [5]:
data_dict = dict()
for a in tree.xpath('//div[contains(@class, "clickventure-node ")]'):
    #print list(a.classes)
    text = a.xpath('.//p')
    #print dir(a.xpath('.//img')[0]), a.xpath('.//img')[0].items()[0][1]
    #print [t.text for t in text]
    allparagraphtext = '\n'.join(t.text.encode('utf-8', 'ignore') for t in text if t.text is not None)
    #print allparagraphtext
    thisid = a.xpath('.//@data-node-id')[0]
    thisdict = {'text': allparagraphtext, 'links' : list()}
    if 'clickventure-node-start' in list(a.classes):
        thisdict['type'] = 'start'
    elif 'clickventure-node-finish' in list(a.classes):
        thisdict['type'] = 'finish'
    else:
        thisdict['type'] = 'normal'
        
    if len(a.xpath('.//img')):
        thisdict['img'] = a.xpath('.//img')[0].items()[0][1]
    else:
        thisdict['img'] = None
    data_dict[int(thisid)] = thisdict
    #print int(thisid[0])
    #print text.text
    for b in a.xpath('.//div[contains(@class, "clickventure-node-link ")]'):
        linkid =  int(b.xpath('.//@data-target-node')[0])
        linktext = b.text_content().strip()
        data_dict[int(thisid)]['links'].append((int(linkid), linktext.encode('ascii', 'ignore')))
        #print b.text_content().strip() + " --> " + str(linkid)
    #print

In [6]:
g = graphviz.Digraph()
for k,v in data_dict.iteritems():
    #print type(v['text'])
    wrappedtext = '\n'.join(textwrap.wrap(v['text'],40))
    if(v['type'] is 'start'):
        style = start_node_style
    elif(v['type'] is 'finish'):
        style = finish_node_style
    else:
        style = normal_node_style
    g.node(str(k), label=wrappedtext.decode('utf-8'),_attributes=style)
    g.graph_attr.update(overall_style)
    #print v
for k,v in data_dict.iteritems():
    for link in v['links']:
        #print link[0]
        wrapped_text = '\n'.join(textwrap.wrap(str(link[1]), 40))
        #print wrapped_text
        g.edge(str(k), str(link[0]), wrapped_text, _attributes=edge_style)

In [7]:
g.render('skull')

'skull.pdf'